In [29]:
# Importing dependencies and libraries
import numpy as np
import json
import requests
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as st
import os
import datetime

from config import api_key


In [32]:
# Looking at stock performance by industry relevant indexes

# Variable initialization
close_daily_price =[]
ticker_date = []
delta = datetime.timedelta(days=1)

# Search parameters
industry_index = ['XPH', 'PCM', 'PDFS', 'PEN', 'PHR']
start_date = datetime.datetime(2019,8,1)
end_date = datetime.datetime(2020,7,31)
base_url = 'https://www.alphavantage.co/query?function=TIME_SERIES_DAILY'

# For loop to create the date column and create the results dataframe
while start_date <= end_date:
    
    ticker_date.append(start_date)
    
    # Building the results dataframe
    ticker_value_df = pd.DataFrame({'Date': ticker_date})
    
    start_date += delta
                                    
# Resetting start date
start_date = datetime.datetime(2019,8,1)
                                    
# For loop to make the call for every industry index and retrieving the data
for ticker in industry_index:
    
    # Building the search url
    url = f'{base_url}&symbol={ticker}&outputsize=full&apikey={api_key}'
    
    # Making the API call
    response = requests.get(url)
    response_json = response.json()
    
    #Progress tracker
    print (f'Processing {ticker} with code {response.status_code}')
    
    # Retrieving daily closing prices
    while start_date <= end_date:
    
        # Obtaining date as string
        string_date = start_date.strftime("%Y-%m-%d")

        # Exception for dates with no operation
        try:
            close_daily_price.append(float(response_json['Time Series (Daily)'][string_date]["4. close"]))
        except (KeyError):
            close_daily_price.append(0)  
        
        start_date += delta
                                    
    # Adding the current ticker to the dataframe
    ticker_value_df[ticker] = pd.Series(close_daily_price)
    close_daily_price.clear()
    start_date = datetime.datetime(2019,8,1)



        

Processing XPH with code 200
Processing PCM with code 200
Processing PDFS with code 200
Processing PEN with code 200
Processing PHR with code 200


In [33]:
# Building the results dataframe

ticker_value_df

,Date,XPH,PCM,PDFS,PEN,PHR
0,2019-08-01,37.95,11.25,13.41,167.81,27.42
1,2019-08-02,37.69,11.37,12.75,162.12,26.86
2,2019-08-03,0.00,0.00,0.00,0.00,0.00
3,2019-08-04,0.00,0.00,0.00,0.00,0.00
4,2019-08-05,36.61,11.27,12.00,157.84,26.44
...,...,...,...,...,...,...
361,2020-07-27,43.39,9.36,20.27,212.94,27.46
362,2020-07-28,43.06,9.34,19.45,213.00,27.52
363,2020-07-29,43.26,9.38,19.61,221.00,28.51
364,2020-07-30,43.31,9.45,24.27,224.53,29.30


In [34]:
clean_df=ticker_value_df.loc[ticker_value_df["XPH"]!=0].reset_index(drop=True)
clean_df

,Date,XPH,PCM,PDFS,PEN,PHR
0,2019-08-01,37.95,11.25,13.41,167.81,27.42
1,2019-08-02,37.69,11.37,12.75,162.12,26.86
2,2019-08-05,36.61,11.27,12.00,157.84,26.44
3,2019-08-06,36.72,11.39,12.01,165.63,27.21
4,2019-08-07,36.77,11.40,12.12,147.02,26.85
...,...,...,...,...,...,...
248,2020-07-27,43.39,9.36,20.27,212.94,27.46
249,2020-07-28,43.06,9.34,19.45,213.00,27.52
250,2020-07-29,43.26,9.38,19.61,221.00,28.51
251,2020-07-30,43.31,9.45,24.27,224.53,29.30


In [35]:
# Pulling data tu obtain the price change %

# Variable initialization
ticker_list = []
initial_price = []
final_price = []

# For loop to retrieve data and build a new dataframe
for column in clean_df:
    if column != 'Date':
        ticker_list.append(column)

        ticker_initial = clean_df.loc[clean_df['Date']=='2019-08-01', column].tolist()
        initial_price.append(ticker_initial[0])
        
        ticker_final = clean_df.loc[clean_df['Date']=='2020-07-31', column].tolist()
        final_price.append(ticker_final[0])

# Building the price change % dataframe
avg_price = pd.DataFrame({'Ticker': ticker_list,
                          'Initial Price': initial_price,
                         'Final Price': final_price})

# Computing price change %
avg_price['Percentage Price Change'] = (avg_price['Final Price'] - avg_price['Initial Price']) / (avg_price['Initial Price'])

#-------------MAPPING-----------------
avg_price['Percentage Price Change'] = (avg_price['Percentage Price Change'] * 100).map("{:.2f}%".format)

avg_price


,Ticker,Initial Price,Final Price,Percentage Price Change
0,XPH,37.95,42.67,12.44%
1,PCM,11.25,9.41,-16.36%
2,PDFS,13.41,24.58,83.30%
3,PEN,167.81,221.91,32.24%
4,PHR,27.42,30.06,9.63%
